tf.keras でcifar10 train

In [2]:
!pwd
import sys
sys.executable

/c/Users/shingo/jupyter_notebook/tfgpu_py36_work/02_keras_py/experiment/01_code_test


'C:\\Users\\shingo\\Anaconda3\\envs\\tfgpu20\\python.exe'

In [1]:
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
y_train, y_test = keras.utils.to_categorical(y_train), keras.utils.to_categorical(y_test)
train_gen = ImageDataGenerator(rescale=1.0/255, rotation_range=18, zoom_range=0.15).flow(X_train, y_train, batch_size=16)
valid_gen = ImageDataGenerator(rescale=1.0/255).flow(X_test, y_test, batch_size=16, shuffle=False)

input_tensor = keras.layers.Input(shape=(32, 32, 3))
model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
x = model.output
x = keras.layers.GlobalAveragePooling2D(name='FC_avg')(x)
predictions = keras.layers.Dense(10, activation='softmax', name='pred')(x)
model = keras.models.Model(inputs=model.input, outputs=predictions)
model.compile(loss=keras.backend.categorical_crossentropy, optimizer='sgd', metrics=['accuracy'])

model.fit_generator(train_gen
                    , steps_per_epoch=X_train.shape[0]//16
                    , epochs=2
                    , validation_data=valid_gen
                    , validation_steps=X_test.shape[0]//16
                    , verbose=1)

Epoch 1/2
3125/3125 [==============================] - 174s 56ms/step - loss: 0.9887 - accuracy: 0.6550 - val_loss: 0.6401 - val_accuracy: 0.7808
Epoch 2/2
3125/3125 [==============================] - 165s 53ms/step - loss: 0.6225 - accuracy: 0.7855 - val_loss: 0.6467 - val_accuracy: 0.7723
